# ShampooSalesTimeSeries

## 1. Introduction and algorithm description
This notebook uses the shampoo sales dataset to demonstrate the time series algorithms below which are provided by the hana_ml. 

- ARIMA
- Auto ARIMA
- Auto Exponential Smoothing
- Seasonal Decompose

### - ARIMA 
The Auto Regressive Integrated Moving Average (ARIMA) algorithm is famous in econometrics, statistics and time series analysis.
There are three integers (p, d, q) that are used to parametrize ARIMA models. Because of that, a nonseasonal ARIMA model is denoted with ARIMA(p, d, q):

 - p is the number of autoregressive terms (AR part). It allows to incorporate the effect of past values into our model. Intuitively, this would be similar to stating that it is likely to be warm tomorrow if it has been warm the past 3 days.
 - d is the number of nonseasonal differences needed for stationarity. Intuitively, this would be similar to stating that it is likely to be same temperature tomorrow if the difference in temperature in the last three days has been very small.
 - q is the number of lagged forecast errors in the prediction equation (MA part). This allows us to set the error of our model as a linear combination of the error values observed at previous time points in the past.

When dealing with seasonal effects, Seasonal ARIMA(SARIMA) is used, which is denoted as ARIMA(p,d,q)(P,D,Q,s). Here, p, d, q are the nonseasonal parameters described above, P, D, Q follow the same definition but are applied to the seasonal component of the time series. The term s is the periodicity of the time series.

### - Auto ARIMA 
Although the ARIMA model is useful and powerful in time series analysis, it is somehow difficult to choose appropriate orders. Hence, auto ARIMA is to determine the orders of an ARIMA model automatically.

### - Auto Exponential Smoothing
Auto exponential smoothing is used to calculate optimal parameters of a set of smoothing functions, including Single Exponential Smoothing, Double Exponential Smoothing, and Triple Exponential Smoothing.

### - Seasonal Decompose
The algorithm is to decompose a time series into three components: seasonal, trend, and random.

## 2. Dataset
Shampoo sales dataset describes the monthly number of sales of shampoo over a 3 year period.
The units are a sales count and there are 36 observations. The original dataset is credited to Makridakis, Wheelwright and Hyndman (1998). We can see that the dataset shows an increasing trend and possibly has a seasonal component. 

<img src="images/Shampoo-Sales.png" title="Temperatures" width="600" height="1200" />


Dataset source: https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv for tutorials use only.

### Attribute information
 - ID: ID
 - SALES: Monthly sales 


## 3. Data Loading

### Import packages
First, import packages needed in the data loading.

In [1]:
from hana_ml import dataframe
from data_load_utils import DataSets, Settings

### Setup Connection
In our case, the data is loaded into a table called "SHAMPOO_SALES_DATA_TBL" in HANA from a csv file "shampoo.csv".
To do that, a connection to HANA is created and then passed to the data loader.
To create a such connection, a config file, <b>config/e2edata.ini</b> is used to control the connection parameters.
A sample section in the config file is shown below which includes HANA url, port, user and password information.  

#########################<br>
[hana]<br>
url=host-url<br>
user=username<br>
passwd=userpassword<br>
port=3xx15<br>
#########################<br>



In [2]:
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = dataframe.ConnectionContext(url, port, user, pwd)

### Load Data
Then, the function DataSets.load_shampoo_data() is used to decide load or reload the data from scratch. If it is the first time to load data, an exmaple of return message is shown below:

##################<br>
ERROR:hana_ml.dataframe:Failed to get row count for the current Dataframe, (259, 'invalid table name:  Could not find table/view SHAMPOO_SALES_DATA_TBL in schema XIN: line 1 col 37 (at pos 36)')
Table SHAMPOO_SALES_DATA_TBL doesn't exist in schema XIN
Creating table SHAMPOO_SALES_DATA_TBL in schema XIN ....
Drop unsuccessful
Creating table XIN.SHAMPOO_SALES_DATA_TBL
Data Loaded:100%
###################<br>

If the data is already loaded, there would be a return message "Table XXX exists and data exists".

In [3]:
data_tbl = DataSets.load_shampoo_data(connection_context)

Table SHAMPOO_SALES_DATA_TBL exists and data exists


### Create Dataframes
Create a dataframe df from SHAMPOO_SALES_DATA_TBL for the following steps.

In [4]:
df = connection_context.table(data_tbl)

### Simple Data Exploration
We will do some data exploration to know the data better.
- First 3 data points

In [5]:
df.collect().head(3)

,ID,SALES
0,25,339.7
1,24,342.3
2,23,264.5


- Columns

In [6]:
print(df.columns)

['ID', 'SALES']


- No. of data points

In [7]:
print('Number of rows in df: {}'.format(df.count()))

Number of rows in df: 36


- Data types

In [8]:
df.dtypes()

[('ID', 'INT', 10, 10, 10, 0), ('SALES', 'DOUBLE', 15, 15, 15, 0)]

## 4. Analysis
In this section, various time series algorithms are applied to analyze the shampoo sales dataset.

### 4.1 Seasonal Decompose
Because the dataset shows an increasing trend and possibly some seasonal component, we first use seasonal decompose function to decompose the data.

In [9]:
from hana_ml.algorithms.pal.tsa.seasonal_decompose import seasonal_decompose

In [10]:
stats, decompose = seasonal_decompose(df, endog= 'SALES', alpha = 0.2, thread_ratio=0.5)

seasonal decompose function returns two tables: stats and decompose.

In [11]:
stats.collect()

,STAT_NAME,STAT_VALUE
0,type,multiplicative
1,period,2
2,acf,0.515912


We could see the data has a seasonality and its period is 2. The corresponding multiplicative seasonality model is identified. The decompose table shows the components.

In [12]:
decompose.collect().head(5)

,ID,SEASONAL,TREND,RANDOM
0,1,1.030443,235.975,1.093935
1,2,0.969557,185.225,0.812423
2,3,1.030443,157.850,1.125693
3,4,0.969557,150.500,0.817581
4,5,1.030443,162.100,1.079416


### 4.2 ARIMA
import the ARIMA module

In [13]:
from hana_ml.algorithms.pal.tsa.arima import ARIMA

Create an ARIMA estimator and make the initialization:

In [14]:
arima = ARIMA(order=(1, 0, 0), seasonal_order=(1, 0, 0, 2),
              method='mle', thread_ratio=1.0)

Perform fit on the given data:

In [15]:
arima.fit(df, endog='SALES')

There are two attributes of ARIMA model: model_ and fitted_. We could see the model parameters in model_. 

In [16]:
arima.model_.collect()

,KEY,VALUE
0,p,1
1,AR,0.0341785
2,d,0
3,q,0
4,MA,
5,s,2
6,P,1
7,SAR,0.881861
8,D,0
9,Q,0


The model_ contains AIC (Akaike Information Criterion) and BIC (Bayes Information Criterion) that can be minimized to select the best fitting model. 

In [17]:
arima.fitted_.collect().set_index('ID').head(5)

,FITTED,RESIDUALS
ID,,
1,312.618416,-46.618416
2,309.623039,-163.723039
3,270.250156,-87.150156
4,162.574295,-43.274295
5,196.818838,-16.518838


Predict uisng the ARIMA model:

In [ ]:
result = arima.predict(forecast_method='innovations_algorithm',forecast_length=5)

In [ ]:
result.collect()

In [ ]:
%matplotlib inline

from hana_ml.visualizers.visualizer_base import forecast_line_plot

ax = forecast_line_plot(pred_data=result.set_index("TIMESTAMP"),
                        confidence=("LO80", "HI80", "LO95", "HI95"),
                        max_xticklabels=10)

### 4.3 Auto ARIMA 
Import auto ARIMA module

In [ ]:
from hana_ml.algorithms.pal.tsa.auto_arima import AutoARIMA

Create an auto ARIMA estimator and make the initialization:

In [ ]:
autoarima = AutoARIMA(search_strategy=1, allow_linear=1, thread_ratio=1.0)

Perform fit on the given data:

In [ ]:
autoarima.fit(df, endog='SALES')

In [ ]:
autoarima.model_.collect()

In [ ]:
autoarima.fitted_.collect().set_index('ID').head(6)

Predict uisng the auto ARIMA model:

In [ ]:
result= autoarima.predict(forecast_method='innovations_algorithm', forecast_length=5)

In [ ]:
result.collect()

### 4.4 Auto Exponential Smoothing 
Import auto exponential smoothing module:

In [ ]:
from hana_ml.algorithms.pal.tsa.exponential_smoothing import AutoExponentialSmoothing

Create an auto exponential smoothing estimator and make the initialization:

In [ ]:
autoexpsmooth = AutoExponentialSmoothing(model_selection=1, forecast_num=3)

Perform the fit on the given data:

In [ ]:
autoexpsmooth.fit_predict(df,endog= 'SALES',)

Have a look at the stats_ and it shows the parameters and Triple Exponential SMoothing (TESM) model is selected.

In [ ]:
autoexpsmooth.stats_.collect()

To see the result of smoothing forecast and upper and lower bound in the forecast_:

In [ ]:
autoexpsmooth.forecast_.collect()

## 5. Close Connection

In [ ]:
connection_context.close()